In [120]:
from datasets import load_dataset
import transformers
import pandas as pd
import numpy as np
import os
import time

In [121]:
import torch
torch.cuda.is_available()
torch.cuda.empty_cache()

In [122]:
data_path = "./dataset/csv/"

## data load

In [123]:
all_data = load_dataset('csv',data_files=os.path.join(data_path,'order_speech_ko1000_000.csv'),sep='\t',split='train')

Using custom data configuration default-0f216d6397fe291f
Reusing dataset csv (C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


## data cleaning

In [124]:
import re
from unicode import split_syllables, join_jamos
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    batch["text"] = split_syllables(batch["text"])
    return batch

In [125]:
remove_spectial_char_data = all_data.map(remove_special_characters)

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-f83e215b608199a6.arrow


## Create Wav2Vec2CTCTokenizer

In [126]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab_jamos.json",
                                 unk_token="<unk>",
                                 pad_token="<pad>",
                                 word_delimiter_token="|")

## Create XLSR-Wav2Vec2 Feature Extractor

In [127]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

In [128]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [129]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [130]:
# processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-ready")

## add audio array

In [131]:
import librosa

In [132]:
sr = 16000

In [133]:
def load_audio(batch):
#     batch['array'],_ = librosa.load('./dataset/audio/'+batch['filename'],sr=16000)
    batch['array'] = np.array(batch['array'][1:-1].split(',')).astype(np.float32)
    return batch

In [134]:
audio_data = remove_spectial_char_data.map(load_audio)

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-6fd2796a774ac563.arrow


In [135]:
type(audio_data[0]['array'])

list

In [136]:
audio_data[0]['array']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.1133459605627838e-12,
 -1.0502000744733175e-12,
 2.2163180578987918e-12,
 -1.5682264514760291e-12,
 -1.5212372919508332e-13,
 4.4247804205366226e-12,
 -1.2672983322470355e-11,
 2.656890679686441e-11,
 -4.7877282000063914e-11,
 7.819626196559071e-11,
 -1.1858666826292108e-10,
 1.6909297573253212e-10,
 -2.2817649159012632e-10,
 2.91835583476896e-10,
 -3.552248772908939e-10,
 4.067994274770115e-10,
 -4.344273552003841e-10,
 4.197922565118972e-10,
 -3.445344287644758e-10,
 1.849441849666178e-10,
 8.215064739580669e-11,
 -4.78959205629792e-10,
 1.023487516960131e-09,
 -1.728527765365584e-09,
 2.5924540292265874e-09,
 -3.6031813088754916e-09,
 4.728607727599865e-09,
 -5.9183116185579365e-09,
 7.0954735420514226e-09,
 -8.165113918323641e-09,
 9.004

## Data augmetation(RIR applied)

In [137]:
rir_raw,_ = librosa.load('./outputs/rir_data.wav',sr)

def rir_applied(batch):
    rir = torch.from_numpy( rir_raw[int(sr * 2.44) : int(sr * 2.7)].reshape(1,-1) )
    rir = rir / torch.norm(rir, p=2)
    rir = torch.flip(rir,[1])

    speech = torch.from_numpy(np.array(batch['array'],dtype=np.float32).reshape(1,-1))

    speech_ = torch.nn.functional.pad(speech, (rir.shape[1] - 1, 0))
    # print(speech.dtype)
    # print(speech_.dtype)
    # print(rir.dtype)
    augmented = torch.nn.functional.conv1d(speech_[None, ...], rir[None, ...])[0]
    batch['array'] = augmented.reshape(-1)
    return batch

def fast_stretching(batch):
    array = np.array(batch['array'],dtype=np.float32)
    batch['array'] = librosa.effects.time_stretch(array,0.8)
    return batch

def slow_stretching(batch):
    array = np.array(batch['array'],dtype=np.float32)
    batch['array'] = librosa.effects.time_stretch(array,1.5)
    return batch

In [138]:
rir_applied_audio_data = audio_data.map(rir_applied)

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-65bbdea21c07bf7c.arrow


In [139]:
fast_stretching_data = audio_data.map(fast_stretching)

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-17315fa06a7e7424.arrow


In [140]:
slow_stretching_data = audio_data.map(slow_stretching)

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-f39a22ddda49e783.arrow


## Preprocess Data

In [141]:
def prepare_dataset(batch):

    # batched output is "un-batched"
    batch["input_values"] = processor(batch["array"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [142]:
ds_list = [audio_data, rir_applied_audio_data, fast_stretching_data, slow_stretching_data]
prepare_ds_list = []
for ds in ds_list:
    prepare_ds_list.append(ds.map(
        prepare_dataset,
        remove_columns=ds.column_names,
        # num_proc=2
    ))

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-7937b1f431298145.arrow
Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-755e215030b240ce.arrow
Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-8efbe5848551afc6.arrow
Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-0f216d6397fe291f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-c785becfa8ad27a7.arrow


In [143]:
augmented_data = torch.utils.data.ConcatDataset(prepare_ds_list)

# Training

## Set-up Trainer

In [144]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.cuda.HalfTensor]]]) -> Dict[str, torch.cuda.HalfTensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [145]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [146]:
from datasets import load_dataset, load_metric, Audio

wer_metric = load_metric("wer")

In [147]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Import Model

In [148]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

loading configuration file https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/config.json from cache at C:\Users\AI_server/.cache\huggingface\transformers\8508c73cd595eb416a1d517b90762416c0bc6cfbef529578079aeae4d8c14336.7581ed2ee0c677f1e933180df51bd1a668c4a2b6d5fd1297d32069373dac097c
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weig

XLSR-Wav2Vec2의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미가 있지만 문맥적으로 독립적인 기능을 추출하는 데 사용되는 CNN 계층 스택으로 구성됩니다.  
모델의 이 부분은 사전 교육 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이 더 이상 미세 조정할 필요가 없습니다. 따라서 특징 추출 부분의 모든 파라미터에 대해 require_grad를 False로 설정할 수 있다.

In [149]:
model.freeze_feature_extractor()

메모리를 절약하기 위해 그라데이션 체크포인팅을 활성화

In [150]:
model.gradient_checkpointing_enable()

## TrainingArguments

In [151]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-ko-demo",
  output_dir="./wav2vec2-large-xlsr-ko-demo",
  group_by_length=True,
  # per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=80,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
#   auto_find_batch_size=True, # need -> pip install accelerate
  load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [152]:
from transformers import Trainer
from torch.utils.data import random_split

ds_size = len(augmented_data)
train_size = int(ds_size*0.8)
val_size = ds_size - train_size
train_ds, val_ds = random_split(augmented_data,[train_size,val_size])

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


## Model Training

In [153]:
trainer.train()

C:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3200
  Num Epochs = 80
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 16000


Step,Training Loss,Validation Loss,Wer
100,5.572800,5.400629,1.000000
200,3.309100,3.387674,1.000000
300,3.261600,3.408355,1.000000
400,3.198900,3.433145,1.000000
500,3.094700,3.183763,1.000000
600,3.047600,3.110659,1.000000
700,2.961600,3.035182,0.997554
800,2.745700,2.687272,0.929305
900,2.166200,2.064660,1.039139
1000,1.735500,1.487775,1.011497


***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-100
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-100\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-100\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-100\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-2800] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-200
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-200\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-200\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-200\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-2900] due to args.save_to

Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-1700\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-1700\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-1700\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-1500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-1800
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-1800\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-1800\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-1800\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-1600] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-dem

Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-3300\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-3300\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-3100] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-3400
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-3400\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-3400\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-3400\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-3200] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-3500
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint

Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-4900\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-4800] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-5000
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-5000\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-5000\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-5000\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-4900] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-5100
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-5100\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-5100\

Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-6300] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-6600
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-6600\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-6600\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-6600\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-6400] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-6700
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-6700\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-6700\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-6700\preproc

***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-8200
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-8200\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-8200\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-8200\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-8000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-8300
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-8300\config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-8300\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-8300\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-8100] due to args

KeyboardInterrupt: 

CTC 손실을 사용하여 더 큰 데이터 세트에서 더 큰 모델을 미세 조정하려면 [여기서](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 공식 음성 인식 예를 살펴봐야 한다.